# Explanation of this notebook: (09/27/2025)

- This is essentially a copy of the 20250909_validate_jpar_calc.ipynb notebook basically line for line, with the exception of two parts (I think):
    - Added some of the relevant columns from both 20250209_joined_events_members.csv and Cumulative JPAR Event Data - Event Information.csv Google sheet
    - Copied calculation_df after the JPAR calculation to reformat some columns to be compatible with existing code of mine.

- There are a few things I need to clean up (e.g., NA's I've ignored, among other issues, but will clean that up later!). Just wanted a working demo.
- The output of this is the input to the dashboard.

In [4]:
# Import required packages
import pandas as pd
import numpy as np
import re

import os


from datetime import datetime

In [5]:
# Read in the data
joined_events_members_filtered = pd.read_csv('../data_jpar/20250209_joined_events_members.csv')
jpar_times = pd.read_csv('../data_jpar/jpar_times.csv')

# remove ppl who joined after the cuttoff
joined_events_members_filtered = joined_events_members_filtered[
    pd.to_datetime(joined_events_members_filtered['initiated_date'], errors='coerce') < pd.Timestamp('2025-02-09')
]

# and exclude connor
# Drop row where full_name == 'conner delaat'
joined_events_members_filtered = joined_events_members_filtered[joined_events_members_filtered['full_name']!= 'conner delaat']


In [6]:
##### ADDED BY JACOB -- GRABBING SOME DATA FOR DASHBOARD
# read in event information (downloaded from google sheets)
event_information_df = pd.read_csv('../dashboard/data/Cumulative JPAR Event Data - Event Information.csv')

# merge some columns onto our dataframe
cols_to_keep = ["Event ID", "Event Name", "Date"]  # whatever you want
joined = joined_events_members_filtered.merge(
    event_information_df[cols_to_keep],
    how="left",
    left_on="event_id",
    right_on="Event ID"
)
# rename 'joined' back to jointed_events_members_filtered 
# for consistency with the rest of the code
joined_events_members_filtered = joined.copy()

# change the columns im including to get more information
calculation_df = (
    joined_events_members_filtered[
        joined_events_members_filtered['membership_status'].isin(['ACTIVE', 'EXPIRED_RECENT'])
    ]
    .copy()
)

In [7]:
# --- Ensure sorting and correct dtypes ---
calculation_df['event_id'] = calculation_df['event_id'].astype(float)
calculation_df = calculation_df.sort_values(['event_id', 'full_name'])

# --- Add event piece count ---
calculation_df['piece_count'] = calculation_df.groupby('event_id')['pieces_assembled'].transform('max')

# --- Compute completion time (seconds) ---
calculation_df['completion_seconds'] = (1 / calculation_df['ppm']) * calculation_df['piece_count'] * 60

# --- Event-level stats ---
event_avg = calculation_df.groupby('event_id')['completion_seconds'].transform('mean')
event_std = calculation_df.groupby('event_id')['completion_seconds'].transform('std')

calculation_df['adj_ind_JPAR'] = calculation_df['completion_seconds'] / event_avg
calculation_df['adj_ind_zscore_JPAR'] = (calculation_df['completion_seconds'] - event_avg) / event_std

# --- Initialize tracking columns ---
calculation_df['previous_JPAR'] = np.nan
calculation_df['JPAR'] = np.nan

# --- Track rolling JPAR per person ---
previous_jpars = {}

for idx, row in calculation_df.iterrows():
    name = row['full_name']
    current_jpar = row['adj_ind_JPAR']

    if name not in previous_jpars:
        # First event for this person
        prev_jpar = np.nan
        jpar = current_jpar
        previous_jpars[name] = [current_jpar]
    else:
        # Average across *all previous* JPARs
        prev_jpar = np.mean(previous_jpars[name])

        # Current JPAR could be the cumulative average including this event
        jpar = np.mean(previous_jpars[name] + [current_jpar])

        # Append this event’s JPAR to the history
        previous_jpars[name].append(current_jpar)

    calculation_df.at[idx, 'previous_JPAR'] = prev_jpar
    calculation_df.at[idx, 'JPAR'] = jpar

In [8]:
##### ADDED BY JACOB ---- FORMATTING FOR EASE OF COMPATABILITY WITH EXISTING CODE I HAVE
# create a new dataframe to match the expected puzzleboard format
output_df = calculation_df.copy()
#### first i need to create a "Rank" for each person in each event;
#### this is just their place in the event. doing this by sorting
#### by completion_seconds, which is the extrapolated time
output_df["Rank"] = (
    output_df.groupby("event_id")["completion_seconds"]
      .rank(method="min", ascending=True)
)
# note that there are a few na's here, but ignoring them for now
###### FIX THIS 
###### FIX THIS 
###### FIX THIS 
###### FIX THIS 
output_df = output_df[output_df["Rank"].notna()]
# add a name column for clean names
output_df["Name"] = output_df["first_name"] + " " + output_df["last_name"]

###### column for Time in HH:MM:SS format,  but have to clean a bit
# make sure completion_seconds is numeric
output_df["completion_seconds"] = pd.to_numeric(output_df["completion_seconds"], errors="coerce")

# find max seconds for each event
output_df["event_max_seconds"] = output_df.groupby("event_id")["completion_seconds"].transform("max")

# function to convert seconds to HH:MM:SS
def seconds_to_hms(sec):
    h = int(sec // 3600)
    m = int((sec % 3600) // 60)
    s = int(sec % 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

# new column 'Time':
# - if not DNF → format actual completion_seconds
# - if DNF     → format event_max_seconds
output_df["Time"] = output_df.apply(
    lambda row: seconds_to_hms(row["completion_seconds"])
    if row["completion_time"] != "DNF"
    else seconds_to_hms(row["event_max_seconds"]),
    axis=1,
)

# add pieces remaining
output_df['Remaining'] = output_df['piece_count']-output_df['pieces_assembled']

# change Date column format
output_df["Date"] = pd.to_datetime(output_df["Date"], format="mixed").dt.strftime("%Y-%m-%d")

# Pieces column for piece_count
output_df['Pieces'] = output_df['piece_count']

# Event column represent the event id for now i suppose
output_df['Event'] = output_df['event_id']

# Full_Event for event name
output_df['Full_Event'] = output_df['Event Name']

# time_in_seconds 
####### NEEDS TO BE CORRECTED
####### NEEDS TO BE CORRECTED
####### NEEDS TO BE CORRECTED
####### NEEDS TO BE CORRECTED
# has different puzzleboard meaning
output_df['time_in_seconds'] = output_df['completion_seconds']

# PTR In = prev_jpar
output_df['PTR In'] = output_df['previous_JPAR']

# PTR Out = JPAR
output_df['PTR Out'] = output_df['JPAR']

# PPM
output_df['PPM'] = output_df['ppm'] 

# corrected_time
output_df['corrected_time'] = output_df['completion_seconds']

# Total Events
output_df["Total Events"] = output_df.groupby("member_id")["event_id"].transform("count")

# rename PTR to JPAR
output_df = output_df.rename(columns={'PTR In': 'JPAR In', 'PTR Out': 'JPAR Out'})

##### Find latest JPAR information
# Sort by member_id and Date so the most recent is last
output_df = output_df.sort_values(['member_id', 'Date'])

# Group by member_id and pick the last (most recent) row for each
latest = output_df.groupby('member_id').last().reset_index()

# Rename the merged columns
latest = latest.rename(columns={
    'JPAR Out': 'Latest JPAR',
    'Full_Event': 'Latest Event',
    'Date': 'Latest Event Date'
})


# Create the new columns in the original dataframe
output_df = output_df.merge(
    latest[['member_id', 'Latest JPAR', 'Latest Event', 'Latest Event Date']],
    on='member_id',
    how='left'
)

In [9]:
# relevant columns
output_df = output_df[['Rank','Name','Time','Remaining','Date','Full_Event',
                       'Latest JPAR', 'Latest Event', 'Latest Event Date',
                       'Pieces','Event','time_in_seconds','JPAR In','member_id',
                       'JPAR Out','PPM','corrected_time','Total Events']]

In [10]:
output_df

,Rank,Name,Time,Remaining,Date,Full_Event,Latest JPAR,Latest Event,Latest Event Date,Pieces,Event,time_in_seconds,JPAR In,member_id,JPAR Out,PPM,corrected_time,Total Events
0,1.0,Tammy McLeod,00:45:11,0.0,2023-09-21,WJPC 2023 First Round D,0.862172,Saint Paul Winter Carnival Solo,2025-01-26,500.0,230921.04,2712.0,NaN,17593433.0,0.693900,11.061947,2712.0,8
1,4.0,Tammy McLeod,00:53:38,0.0,2023-09-22,WJPC 2023 Semifinal 2,0.862172,Saint Paul Winter Carnival Solo,2025-01-26,500.0,230922.02,3218.0,0.693900,17593433.0,0.767208,9.322561,3218.0,8
2,14.0,Tammy McLeod,00:59:09,0.0,2023-09-23,WJPC 2023 Final,0.862172,Saint Paul Winter Carnival Solo,2025-01-26,500.0,230923.00,3550.0,0.767208,17593433.0,0.830374,8.450704,3550.0,8
3,4.0,Tammy McLeod,00:47:04,0.0,2024-09-19,WJPC 2024 First Round D,0.862172,Saint Paul Winter Carnival Solo,2025-01-26,500.0,240919.01,2824.0,0.830374,17593433.0,0.823650,10.623229,2824.0,8
4,9.0,Tammy McLeod,00:48:26,0.0,2024-09-20,WJPC 2024 Semifinal 2,0.862172,Saint Paul Winter Carnival Solo,2025-01-26,500.0,240920.02,2906.0,0.823650,17593433.0,0.861196,10.323469,2906.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2462,134.0,Stephanie Klenzing,01:16:12,0.0,2025-02-08,Speed Puzzling # 292,1.212010,Speed Puzzling # 292,2025-02-08,500.0,250208.00,4572.0,1.071919,24098367.0,1.212010,6.561680,4572.0,2
2463,16.0,Christie Dizzia,02:42:03,0.0,2025-01-10,FL2025 - Florida Speed Puzzling Championship,1.543276,FL2025 - Florida Speed Puzzling Championship,2025-01-10,500.0,250110.01,9723.0,NaN,24099749.0,1.543276,3.085467,9723.0,1
2464,27.0,Jennifer McGinnis,01:34:29,0.0,2025-02-01,TX2025 - Texas Speed Puzzling Championship,0.967167,TX2025 - Texas Speed Puzzling Championship,2025-02-01,500.0,250201.00,5669.0,NaN,24108256.0,0.967167,5.291939,5669.0,1
2465,6.0,Melinda Shokler,01:03:14,0.0,2025-02-01,TX2025 - Texas Speed Puzzling Championship,0.647280,TX2025 - Texas Speed Puzzling Championship,2025-02-01,500.0,250201.00,3794.0,NaN,24108422.0,0.647280,7.907222,3794.0,1


In [25]:
output_df.to_pickle('./data/test_pickle.pkl')

In [3]:
!ls

README.md                  notes.txt
data                       requirements.txt
jpar_recalc_notebook.ipynb streamlit_app.py
nav                        utils


# Puzzleboard Format Comparison

In [9]:
data = pd.read_pickle('../dashboard/data/250811_scrape.pkl')
# get only the 500 pieces and those with data entry error
data = data[(data['Pieces'] == 500) | (data['Pieces'].isna())]

In [39]:
print(data)

       Rank                      Name     Time  \
0       348     Alfredo Sánchez Gómez  2:30:00   
1       303     Alba Erustes González  2:30:00   
2       290    Beatriz Luengo Tejedor  2:30:00   
3       352    Alba Sánchez Fernández  2:30:00   
4       304  Alba Hernández Hernández  2:30:00   
...     ...                       ...      ...   
32227   152              Toni Fussell  2:30:00   
32228   153            Alannah Gillis  2:30:00   
32229   154            Maegen Purcell  2:30:00   
32230   155             Michelle Chan  2:30:00   
32231   156      Sally Garmony-Burton  2:30:00   

                                        Remaining       Date  Pieces  \
0      201.00000000000000000000000000000000000000 2024-06-15     500   
1       47.00000000000000000000000000000000000000 2024-06-15     500   
2        5.00000000000000000000000000000000000000 2024-06-15     500   
3      240.00000000000000000000000000000000000000 2024-06-15     500   
4       50.0000000000000000000000000000

In [11]:
data.columns

Index(['Rank', 'Name', 'Time', 'Remaining', 'Date', 'Pieces', 'Event',
       'Full_Event', 'time_in_seconds', 'PTR In', 'PTR Out',
       'Avg PTR In (Event)', '12-Month Avg Completion Time', 'PPM',
       'Latest JPAR Out', 'time_penalty', 'corrected_time', 'Total Events'],
      dtype='object')

In [38]:
calculation_df.columns

Index(['full_name', 'member_id', 'completion_time', 'event_id',
       'pieces_assembled', 'max_time_seconds', 'ppm', 'first_name',
       'last_name', 'mp_id_deduped', 'membership_cutoff_date',
       'membership_status', 'mp_id', 'initiated_date', 'match_type',
       'Event ID', 'Event Name', 'Date', 'piece_count', 'completion_seconds',
       'adj_ind_JPAR', 'adj_ind_zscore_JPAR', 'previous_JPAR', 'JPAR'],
      dtype='object')